# 2.

data prep for ML training. requires ```db_data_scraper.ipynb```

In [1]:
import sqlite3
import pandas as pd
import datetime
import numpy as np
from matplotlib import pyplot as plt
import time

database = "../main.db"

In [2]:
# conn.close() # comment this if running the first time
conn = sqlite3.connect(database)

## get data from the db

select only those combinations (ticker,earnings_date) where the EPS actual was positive.

In [6]:
q = """
    SELECT
        *
    FROM yf_secondary_expanded
    WHERE eps_diff > 0.0
"""
basic=pd.read_sql_query(q,conn).set_index(['ticker','earnings_date'])

In [7]:
basic

financial_date  rnk  eps_est  eps_actual  eps_diff  \
ticker earnings_date                                                      
A      2021-05-25        2020-10-31    1     0.83        0.97      0.14   
       2021-08-17        2020-10-31    1     0.99        1.10      0.11   
       2021-11-22        2021-10-31    1     1.18        1.21      0.03   
       2022-02-22        2021-10-31    1     1.18        1.21      0.03   
       2022-05-24        2021-10-31    1     1.12        1.13      0.01   
...                             ...  ...      ...         ...       ...   
ZTS    2022-05-05        2021-12-31    1     1.23        1.32      0.09   
ZUO    2021-12-01        2021-01-31    1    -0.03       -0.02      0.01   
       2022-03-02        2022-01-31    1    -0.02       -0.01      0.01   
       2022-08-24        2022-01-31    1    -0.05       -0.03      0.02   
       2022-12-06        2022-01-31    1    -0.06       -0.02      0.04   

                      eps_diff_pct    tot_assets  common_stock       tot_cap  \
ticker earnings_date                                                           
A      2021-05-25           0.1729  9.627000e+09     3000000.0  7.157000e+09   
       2021-08-17           0.1089  9.627000e+09     3000000.0  7.157000e+09   
       2021-11-22           0.0298  1.070500e+10     3000000.0  8.118000e+09   
       2022-02-22           0.0289  1.070500e+10     3000000.0  8.118000e+09   
       2022-05-24           0.0089  1.070500e+10     3000000.0  8.118000e+09   
...                            ...           ...           ...           ...   
ZTS    2022-05-05           0.0706  1.390000e+10     5000000.0  1.113500e+10   
ZUO    2021-12-01           0.3103  4.215070e+08       12000.0  1.735790e+08   
       2022-03-02           0.5833  4.412520e+08       13000.0  1.706070e+08   
       2022-08-24           0.4000  4.412520e+08       13000.0  1.706070e+08   
       2022-12-06           0.6552  4.412520e+08       13000.0  1.706070e+08   

                          tot_debt  ...  expense_to_income  assets_to_profit  \
ticker earnings_date                ...                                        
A      2021-05-25     2.359000e+09  ...           2.353428          3.393373   
       2021-08-17     2.359000e+09  ...           2.353428          3.393373   
       2021-11-22     2.729000e+09  ...           1.529324          3.142060   
       2022-02-22     2.729000e+09  ...           1.529324          3.142060   
       2022-05-24     2.729000e+09  ...           1.529324          3.142060   
...                            ...  ...                ...               ...   
ZTS    2022-05-05     6.743000e+09  ...           0.952551          2.539741   
ZUO    2021-12-01     6.928300e+07  ...          -3.364545          2.413438   
       2022-03-02     5.875500e+07  ...          -3.148478          2.135449   
       2022-08-24     5.875500e+07  ...          -3.148478          2.135449   
       2022-12-06     5.875500e+07  ...          -3.148478          2.135449   

                      debt_to_debt  debt_to_cap  profit_to_reve  \
ticker earnings_date                                              
A      2021-05-25         0.389148     0.329607        0.531373   
       2021-08-17         0.389148     0.329607        0.531373   
       2021-11-22         0.456211     0.336167        0.539168   
       2022-02-22         0.456211     0.336167        0.539168   
       2022-05-24         0.456211     0.336167        0.539168   
...                            ...          ...             ...   
ZTS    2022-05-05         0.461219     0.605568        0.703832   
ZUO    2021-12-01              NaN     0.399144        0.571836   
       2022-03-02              NaN     0.344388        0.595931   
       2022-08-24              NaN     0.344388        0.595931   
       2022-12-06              NaN     0.344388        0.595931   

                      on_earnings_close  on_earnings_volume    market_cap  \
ticker earning

In [8]:
hist_q = """
    SELECT 
        ticker,
        DATE(trade_date) trade_date,
        close
    FROM yf_history 
"""
hist  = pd.read_sql_query(hist_q,conn)

In [9]:
hist

,ticker,trade_date,close
0,A,2019-01-31,76.050003
1,A,2019-02-01,76.040001
2,A,2019-02-04,76.029999
3,A,2019-02-05,76.309998
4,A,2019-02-06,76.300003
...,...,...,...
1020411,KBH,2023-01-13,35.099998
1020412,KBH,2023-01-17,35.189999
1020413,KBH,2023-01-18,35.290001
1020414,KBH,2023-01-19,34.799999


In [16]:
earnings_date = '2022-06-01'
lookback_weeks = 4*3

sub_start = (datetime.datetime.strptime(earnings_date, "%Y-%m-%d") - datetime.timedelta(weeks=lookback_weeks)).strftime("%Y-%m-%d")
sub_start

'2022-03-09'

In [20]:
def get_history_from_prev_period(ticker, earnings_date, lookback_weeks = 4*3): # this is for past
    # get prev TODO
    sub_start = (datetime.datetime.strptime(earnings_date, "%Y-%m-%d") - datetime.timedelta(weeks=lookback_weeks)).strftime("%Y-%m-%d")
    sub_hist  = hist[(hist.ticker==ticker) & (hist.trade_date>=sub_start) & (hist.trade_date<earnings_date)].sort_values('trade_date', ascending=True)
    if sub_hist.shape[0]==0:
        return np.nan
    price_var = sub_hist.close.std()/sub_hist.close.mean()
    return price_var

def get_history_for(ticker, earnings_date): # this is for future !
    sub_basic = basic.loc[(ticker, earnings_date)]
    sub_start = sub_basic['history_start_date'][0]
    sub_end   = sub_basic['history_end_date'][0]
    sub_hist  = hist[(hist.ticker==ticker) & (hist.trade_date>=sub_start) & (hist.trade_date<=sub_end)].sort_values('trade_date', ascending=True)
    if sub_hist.shape[0]==0:
        return -1,-1
    init_close = sub_hist.head(1)['close'].values[0]
    sub_hist['price_pct'] = sub_hist.close/init_close-1.0
    return sub_hist['price_pct'].min(), sub_hist['price_pct'].max()

# price_min, price_max = get_history_for('ZUO', '2022-12-06')

In [22]:
i = 0
for ticker, earnings_date in basic.index[:]:
    print(i, ticker, earnings_date)
    price_min, price_max = get_history_for(ticker, earnings_date)
    price_var = get_history_from_prev_period(ticker, earnings_date)
    basic.loc[(ticker, earnings_date),'price_min'] = price_min
    basic.loc[(ticker, earnings_date),'price_max'] = price_max
    basic.loc[(ticker, earnings_date),'price_var'] = price_var
    i+=1

0 A 2021-05-25
1 A 2021-08-17
2 A 2021-11-22
3 A 2022-02-22
4 A 2022-05-24
5 A 2022-08-16
6 A 2022-11-21
7 AA 2021-10-14
8 AA 2022-01-19
9 AA 2022-04-20
10 AA 2022-07-20
11 AA 2023-01-18
12 AAL 2021-07-22
13 AAL 2021-10-21
14 AAL 2022-01-20
15 AAL 2022-04-21
16 AAL 2022-10-20
17 AAON 2021-05-06
18 AAON 2021-08-05
19 AAON 2022-05-05
20 AAON 2022-11-07
21 AAPL 2021-07-27
22 AAPL 2022-01-27
23 AAPL 2022-04-28
24 AAPL 2022-07-28
25 AAPL 2022-10-27
26 ABBV 2021-07-30
27 ABBV 2021-10-29
28 ABBV 2022-02-02
29 ABBV 2022-04-29
30 ABBV 2022-07-29
31 ABBV 2022-10-28
32 ABC 2021-08-04
33 ABC 2021-11-04
34 ABC 2022-05-04
35 ABC 2022-08-03
36 ABC 2022-11-03
37 ABCL 2022-05-10
38 ABCL 2022-11-08
39 ABM 2021-06-08
40 ABM 2021-09-08
41 ABM 2021-12-15
42 ABM 2022-03-08
43 ABM 2022-06-08
44 ABM 2022-09-09
45 ABM 2022-12-13
46 ABMD 2020-08-06
47 ABMD 2020-10-29
48 ABMD 2021-01-28
49 ABMD 2021-04-29
50 ABMD 2021-08-04
51 ABMD 2021-10-28
52 ABMD 2022-02-03
53 ABMD 2022-04-28
54 ABMD 2022-08-04
55 ABMD 2022-

In [23]:
assert basic.price_min.isnull().sum()==0

In [24]:
basic.to_csv('raw_features.csv')